In [1]:
import win32gui
import win32api
import win32con
import pickle
import glob
import time

import tensorflow as tf
import numpy as np

from PIL import Image, ImageDraw, ImageGrab
from skimage import io

from random import randint, uniform, sample, seed


from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D 
from keras.models import Model, load_model

import keras.backend as K

Using TensorFlow backend.


### Вспомогательные и тестовые функции

In [2]:
def callback(hwnd, extra):
    """
    Detects position and size of the Vysor application window
    Example copied from Stackoverflow
    Input:
    - hwnd: unknown
    - extra: nothing
    Output: nothing
    """
    rect = win32gui.GetWindowRect(hwnd)
    x = rect[0]
    y = rect[1]
    w = rect[2] - x
    h = rect[3] - y
    
    win_name = win32gui.GetWindowText(hwnd)
    
    if (win_name == "FRD L19"):
        print("Window %s:" % win_name)
        print("\tLocation: (%d, %d)" % (x, y))
        print("\t    Size: (%d, %d)" % (w, h))
        
        if ((x >= 0) and (y >= 0)):
            start_box_left_top_x = x + 260
            start_box_left_top_y = y + 680

            start_box_right_bottom_x = start_box_left_top_x + 45
            start_box_right_bottom_y = start_box_left_top_y + 55

            #h_x, h_y = win32api.GetCursorPos()
            #print(h_x, h_y)

            #im = pyscreenshot.grab(bbox=(start_box_left_top_x, start_box_left_top_y, start_box_right_bottom_x, start_box_right_bottom_y))
            #im.show()
            #im.save("GameStarter.png")

win32gui.EnumWindows(callback, None)

### Основные функции

In [17]:
# Окно приложения, левый верхний угол: 663 0
# Центр блюда в ряду 1, колонке 1: 727 350

def fix_window_coordinates_and_size(window_name, use_external_screen=True):
    """
    Returns coordinates of the left top corner of the window
    Input:
    - window_name: target window name, string
    Output:
    - x: x coordinate, int
    - y: y coordinate, int
    - w: window width (over the X axis)
    - h: window height (over the Y axis)
    """
    
    # Get window handle
    win_handle = win32gui.FindWindow(None, window_name)
    
    assert win_handle, "Window not found!"

    # Get window coordinates
    rect = win32gui.GetWindowRect(win_handle)
    x = rect[0]
    y = rect[1]
    w = rect[2] - x
    h = rect[3] - y

    if (use_external_screen):
        win32gui.MoveWindow(win_handle, 600, 0, 380, 760, True)
        result = (600, 0, 380, 760)
    else:
        win32gui.MoveWindow(win_handle, 900, 0, 380, 760, True)
        result = (900, 0, 380, 760)
    
    return result


def check_if_new_game(x, y, parameters):
    """
    Checks if there is a new game screen with the start new game triangle
    Input:
    - x: left top corner x coordinate of the application, int
    - y: left top corner y coordinate of the application, int
    - parameters: dict with parameters incl. Game_Starter ethalon box, dict
    Output:
    - result: boolean. True if there is a new game screen, False otherwise
    """

    # Get part of the window, where the game starter circle/triangle is expected
    game_starter = ImageGrab.grab(bbox=(x + 172, y + 481, x + 209, y + 519))
    game_starter = game_starter.convert("L")
    game_starter = np.array(game_starter)
    
    # Load ethalon game starter box
    ethalon = parameters["Game_Starter"]
    
    # Check them
    tmp2 = np.abs(game_starter.astype(int) - ethalon.astype(int))
    difference = tmp2.sum()    
    
    if (difference < 300.):
        print("We can start a new game")
        result = True
    else:
        print("Game is progress, difference =", difference)
        result = False
    
    return result


def press_start(x, y):
    """
    Presses the Start New Game triangle
    Input:
    - x: top left corner x coordinate of the window, int
    - y: top left corner y coordinate of the window, int
    Output:
    - True
    """

    x_res = win32api.GetSystemMetrics(0)
    y_res = win32api.GetSystemMetrics(1)
    
    start_x = x + 172 + 19
    start_y = y + 481 + 19
    
    nx = int(start_x * 65535 / x_res)
    ny = int(start_y * 65535 / y_res)
    
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE | win32con.MOUSEEVENTF_MOVE, nx, ny, 0, 0)
    win32api.Sleep(200)
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE | win32con.MOUSEEVENTF_LEFTDOWN, nx, ny, 0, 0)
    win32api.Sleep(200)
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE | win32con.MOUSEEVENTF_LEFTUP, nx, ny, 0, 0)
    win32api.Sleep(500)
    
    return True


def press_next_game(x, y):
    """
    Presses the Play Again button
    Input:
    - x: top left corner x coordinate of the window, int
    - y: top left corner y coordinate of the window, int
    Output:
    - True
    """

    x_res = win32api.GetSystemMetrics(0)
    y_res = win32api.GetSystemMetrics(1)
    
    start_x = x + 180
    start_y = y + 480
    
    nx = int(start_x * 65535 / x_res)
    ny = int(start_y * 65535 / y_res)
    
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE | win32con.MOUSEEVENTF_MOVE, nx, ny, 0, 0)
    win32api.Sleep(200)
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE | win32con.MOUSEEVENTF_LEFTDOWN, nx, ny, 0, 0)
    win32api.Sleep(200)
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE | win32con.MOUSEEVENTF_LEFTUP, nx, ny, 0, 0)
    win32api.Sleep(500)
    
    return True


def get_1_digit_score(score, parameters):
    """
    Extracts one digit score (3-9)
    Input:
    - score: PIL Image with the score
    - parameters: dict with parameters incl. ethalon digits, dict
    Output:
    - score, positive if score found, int. -1 otherwise.
    """
    # Load ethalon large digits
    ethalon_single = parameters["Ethalon_Single"]
    
    # Crop the digit
    digit = np.array(score.crop((13, 9, 23, 25)))    
    
    # Find the best possible ethalon digit
    tmp2 = np.abs(digit.astype(int) - ethalon_single.astype(int))
    result = tmp2.sum(axis=(1, 2))
    
    if (np.min(result) > 500.):
        choice = -1
    else:
        choice = np.argmin(result)    
    
    return choice


def get_2_digits_score(score, parameters):
    """
    Extracts two digits score (xx)
    Input:
    - score: PIL Image with the score
    - parameters: dict with parameters incl. ethalon digits, dict
    Output:
    - score, positive if score found, int. -1 otherwise.
    """
    
    # Load ethalon large digits
    ethalon_large_left = parameters["Ethalon_Large_Left"]
    ethalon_large_right = parameters["Ethalon_Large_Right"]

    # Crop the digits
    left_digit = np.array(score.crop((6, 9, 16, 25) ))
    right_digit = np.array(score.crop((19, 9, 29, 25)))
    
    # Find the best possible left ethalon digit
    tmp2 = np.abs(left_digit.astype(int) - ethalon_large_left.astype(int))
    result = tmp2.sum(axis=(1, 2))
    
    if (np.min(result) > 500.):
        return -1
    
    left_choice = np.argmin(result)
    
    # Find the best possible right ethalon digit
    tmp2 = np.abs(right_digit.astype(int) - ethalon_large_right.astype(int))
    result = tmp2.sum(axis=(1, 2))
    
    right_choice = np.argmin(result)
    
    return left_choice*10 + right_choice


def get_3_digits_score(score, parameters):
    """
    Extracts three digits score (xxx)
    Input:
    - score: PIL Image with the score
    - parameters: dict with parameters incl. ethalon digits, dict
    Output:
    - score, positive if score found, int. -1 otherwise.
    """
    # Load ethalon large digits
    ethalon_small_left = parameters["Ethalon_Small_Left"]
    ethalon_small_middle = parameters["Ethalon_Small_Middle"]
    ethalon_small_right = parameters["Ethalon_Small_Right"]
    
    # Crop the digits
    left_digit = np.array(score.crop((4, 12, 12, 23)))
    middle_digit = np.array(score.crop((14, 12, 22, 23)))
    right_digit = np.array(score.crop((23, 12, 31, 23)))
    
    # Find the best possible left ethalon digit
    tmp2 = np.abs(left_digit.astype(int) - ethalon_small_left.astype(int))
    result = tmp2.sum(axis=(1, 2))
    
    if (np.min(result) > 500.):
        return -1
    
    left_choice = np.argmin(result)
    
    # Find the best possible middle ethalon digit
    tmp2 = np.abs(middle_digit.astype(int) - ethalon_small_middle.astype(int))
    result = tmp2.sum(axis=(1, 2))
    middle_choice = np.argmin(result)
    
    # Find the best possible right ethalon digit
    tmp2 = np.abs(right_digit.astype(int) - ethalon_small_right.astype(int))
    result = tmp2.sum(axis=(1, 2))
    right_choice = np.argmin(result)
    
    return left_choice*100 + middle_choice*10 + right_choice


def get_current_score(x, y, parameters):
    """
    Gets current score. Wrapper for calls to get_1_digit_score, get_2_digits_score, get_3_digits_score
    Input:
    - x: top left corner x coordinate of the application
    - y: top left corner y coordinate of the application
    - parameters: dict with parameters incl. ethalon digits, dict
    Output:
    - score, int
    """
    
    # Get the score plate
    img_score = ImageGrab.grab(bbox=(x + 269, y + 654, x + 304, y + 689))
    img_bw = img_score.convert("L")
    
    # Get the score based on the number of digits
    three_digits_score = get_3_digits_score(img_bw, parameters)
    
    if (three_digits_score == -1):
        two_digits_score = get_2_digits_score(img_bw, parameters)
        
        if (two_digits_score == -1):
            score = get_1_digit_score(img_bw, parameters)
        else:
            score = two_digits_score
    else:
        score = three_digits_score
    
    return score


def process_move_142(move):
    """
    There are total 142 possible moves:
    - along the vertical axis: 6 + 5*12 + 6 = 6*12 = 72
    - along the norizontal axis: 7 + 4*14 + 7 = 5*14 = 70
    
    The _move_ parameter:
    Vertical moves:
    move in 1-6: row 1, column _move_, down
    move in 7-66: row (move - 7 // 12) + 2, column (((move - 7) % 12) // 2) + 1, direction _move_ % 2 == 0 - up, otherwise - down
    move in 67-72: row 7, column (move - 66), up
    
    Horizontal moves:
    move in 73-79: row (move - 72), column 1, right
    move in 80-135: row ((move - 80) % 14) // 2 + 1, column ((move - 80) // 14) + 2, direction _move_ % 2 == 0 - left, otherwise - right
    move in 136-142: row (move - 135), column 6, left    
    
    Input:
    - move: move code, 1-142, int
    Output:
    - row: row where the moved object located, 1-7, int
    - column: row where the moved object located, 1-6, int
    - direction: "up"/"down"/"left"/"right", string
    """
    # Process move
    if (move in range(1, 7)):
        row = 1
        column = move
        direction = "down"
    elif (move in range(7, 67)):
        row = ((move - 7) // 12) + 2
        column = (((move - 7) % 12) // 2) + 1
        direction = "up" if (move % 2 == 0) else "down"
    elif (move in range(67, 73)):
        row = 7
        column = move - 66
        direction = "up"
    elif (move in range(73, 80)):
        row = move - 72
        column = 1
        direction = "right"
    elif (move in range(80, 136)):
        row = (((move - 80) % 14) // 2) + 1
        column = ((move - 80) // 14) + 2
        direction = "left" if (move % 2 == 0) else "right"
    else:
        row = move - 135
        column = 6
        direction = "left"
    
    return row, column, direction


def process_move_71(move):
    """
    There are total 71 possible moves:
    - along the vertical axis: 6*6 = 36
    - along the horizontal axis: 7*5 = 35
    - Total 35 + 36 = 71
    
    The _move_ parameter:
    Vertical moves:
    move in 1-36: row (move - 1) % 6 + 1, column (move - 1) // 6 + 1, "down"
    
    Horizontal moves:
    move in 37-71: row (move - 37) // 5 + 1, column (move - 37) % 5 + 1, "right"
    
    Input:
    - move: move code, 1-66, int
    Output:
    - row: row where the moved object located, 1-7, int
    - column: row where the moved object located, 1-6, int
    - direction: "down"/"right", string
    """
    # Process move
    if (move in range(1, 37)):
        row = (move - 1) % 6 + 1
        column = (move - 1) // 6 + 1
        direction = "down"
    else:
        # move in 37-71
        column = (move - 37) % 5 + 1
        row = (move - 37) // 5 + 1
        direction = "right"
    
    return row, column, direction


def make_move(x, y, number_of_moves, move):
    """
    TODO
    """
    if (number_of_moves == 142):
        row, column, direction = process_move_142(move)
    else:
        row, column, direction = process_move_71(move)
    
    plate_size = 58
    first_plate_center_x = x + 18 + 29
    first_plate_center_y = y + 229 + 29
    
    horizontal_margin = 0
    vertical_margin = 4

    start_x = first_plate_center_x + (column - 1)*(plate_size + horizontal_margin)
    start_y = first_plate_center_y + (row - 1)*(plate_size + vertical_margin)
    
    if (direction == "down"):
        end_x = start_x
        end_y = start_y + plate_size + vertical_margin
    elif (direction == "up"):
        end_x = start_x
        end_y = start_y - plate_size - vertical_margin
    elif (direction == "left"):
        end_x = start_x - plate_size - horizontal_margin
        end_y = start_y 
    else:
        end_x = start_x + plate_size + horizontal_margin
        end_y = start_y 

    # Now handle the mouse (drag with the left button pressed)
    x_res = win32api.GetSystemMetrics(0)
    y_res = win32api.GetSystemMetrics(1)
    
    nx = int(start_x * 65535 / x_res)
    ny = int(start_y * 65535 / y_res)
    
    n_x_dist = int((end_x - start_x) * 65535 / x_res)
    n_y_dist = int((end_y - start_y) * 65535 / y_res)
    
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE | win32con.MOUSEEVENTF_MOVE, nx, ny, 0, 0)
    win32api.Sleep(300)
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE | win32con.MOUSEEVENTF_LEFTDOWN, nx, ny, 0, 0)
    win32api.Sleep(150)
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE | win32con.MOUSEEVENTF_MOVE, nx + n_x_dist, ny + n_y_dist, 0, 0)
    win32api.Sleep(150)
    win32api.mouse_event(win32con.MOUSEEVENTF_ABSOLUTE | win32con.MOUSEEVENTF_LEFTUP, nx + n_x_dist, ny + n_y_dist, 0, 0)
    win32api.Sleep(300)
        
    return


#
# Pixelization v1.0 Simple (no bonus plates detection)
# Готовим трехмерный массив из 0 и 1 размера 7x6x4
# Каждый из последних четырех слоёв отвечает за свой цвет
# Массив цветов в таком порядке: [red, green, blue, purple]
# по принципу RGB+Purple
#
def pixelization_v1(x, y, parameters):
    """
    Converts picture with plates into 3D matrix [7, 6, 4]
    Input:
    - x: top left corner x coordinate of the application
    - y: top left corner y coordinate of the application
    - parameters: dict with parameters incl. ethalon colors, dict
    Output:
    - np.array, shape = [7, 6, 4]
    """
    # Initialize all required parameters
    ethalon_colors = parameters["Ethalon_Colors"]
    
    plate_size = 58
    horizontal_margin = 0
    vertical_margin = 4
    
    result = np.zeros((7, 6, 4))
    
    # Get part of the image with plates
    plates = ImageGrab.grab(bbox=(x + 18, y + 229, x + 361, y + 658))

    for ii in range(7):
        target_y = ii*(plate_size + vertical_margin) + 3
        
        for jj in range(6):
            target_x = jj*(plate_size + horizontal_margin - 1) + 29
            current_color = np.array(plates.crop((target_x, target_y, target_x + 1, target_y + 1)))
            
            tmp = np.abs((ethalon_colors - current_color.astype(int))).sum(axis=(1, 2, 3))
            result[ii, jj, np.argmin(tmp)] = 1

    return result


#
# Pixelization v2.0 Simple (no bonus plates detection)
# Готовим двумерный массив размера 7x6x1, состоящий из 0.25, 0.5, 0.75, 1.0 [red, green, blue, purple]
# Массив цветов в таком порядке: [red, green, blue, purple]
# по принципу RGB+Purple
#
def pixelization_v2(x, y, parameters):
    """
    Converts picture with plates into 2D matrix [7, 6]
    Input:
    - x: top left corner x coordinate of the application
    - y: top left corner y coordinate of the application
    - parameters: dict with parameters incl. ethalon colors, dict
    Output:
    - np.array, shape = [7, 6]
    """
    # Initialize all required parameters
    ethalon_colors = parameters["Ethalon_Colors"]
    
    plate_size = 58
    horizontal_margin = 0
    vertical_margin = 4
    
    result = np.zeros((7, 6, 1))
    
    # Get part of the image with plates
    plates = ImageGrab.grab(bbox=(x + 18, y + 229, x + 361, y + 658))

    for ii in range(7):
        target_y = ii*(plate_size + vertical_margin) + 3
        
        for jj in range(6):
            target_x = jj*(plate_size + horizontal_margin - 1) + 29
            current_color = np.array(plates.crop((target_x, target_y, target_x + 1, target_y + 1)))
            
            tmp = np.abs((ethalon_colors - current_color.astype(int))).sum(axis=(1, 2, 3))
            result[ii, jj, 0] = (np.argmin(tmp) + 1) / 4.0

    return result


def Aero_CNN_v1(input_shape = (7, 6, 4), classes = 71):
    """
    Keras CNN
    INPUT -> CONV -> RELU -> CONV -> RELU -> FC -> RELU -> OUTPUT
    """
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((1, 1))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
    X = Activation('relu')(X)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv1')(X)
    X = Activation('relu')(X)
    
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(classes, activation='relu', name='fc0')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='Aero_CNN')

    return model


def Aero_CNN_v2(input_shape = (7, 6, 1), classes = 142):
    """
    Keras CNN
    INPUT -> CONV -> RELU -> CONV -> RELU -> FC -> RELU -> OUTPUT
    """
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((2, 2))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(16, (5, 5), strides = (1, 1), name = 'conv0')(X)
    X = Activation('relu')(X)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv1')(X)
    X = Activation('relu')(X)
    
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(classes, activation='relu', name='fc0')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='Aero_CNN')

    return model


def Aero_CNN_v3(input_shape = (7, 6, 1), classes = 71):
    """
    Keras CNN
    INPUT -> CONV -> RELU -> CONV -> RELU -> FC -> RELU -> OUTPUT
    """
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((1, 1))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
    X = Activation('relu')(X)
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv1')(X)
    X = Activation('relu')(X)
    
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(classes, activation='relu', name='fc0')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='Aero_CNN')

    return model


def Aero_CNN_v4(input_shape = (7, 6, 1), classes = 71):
    """
    Keras CNN
    INPUT -> CONV -> RELU -> CONV -> RELU -> FC -> RELU -> OUTPUT
    """
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((1, 1))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
    X = Activation('relu')(X)
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv1')(X)
    X = Activation('relu')(X)
    
    # FLATTEN X (means convert it to a vector) + FC + RELU
    X = Flatten()(X)
    X = Dense(256, name='fc0')(X)
    X = Activation('relu')(X)
    
    # FULLYCONNECTED
    X = Dense(classes, name='fc1')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='Aero_CNN')

    return model


def replay_memory_converter(replay_memory):
    """
    Replay memory converter from 4D to 1D
    TODO
    """
    result = []
    
    for item in replay_memory:
        s_before = item[0]

        s_before[:, :, 0] = s_before[:, :, 0] * 0.25
        s_before[:, :, 1] = s_before[:, :, 1] * 0.5
        s_before[:, :, 2] = s_before[:, :, 2] * 0.75
        s_before[:, :, 3] = s_before[:, :, 3] * 1.0

        s_before = s_before.sum(axis=2)

        s_before = np.expand_dims(s_before, axis=2)
        
        s_after = item[3]

        s_after[:, :, 0] = s_after[:, :, 0] * 0.25
        s_after[:, :, 1] = s_after[:, :, 1] * 0.5
        s_after[:, :, 2] = s_after[:, :, 2] * 0.75
        s_after[:, :, 3] = s_after[:, :, 3] * 1.0

        s_after = s_after.sum(axis=2)

        s_after = np.expand_dims(s_after, axis=2)      
        
        result.append( (s_before, item[1], item[2], s_after) )
        
    return result

### Main program body

#### Эталонные параметры
Размер окна, чтобы работало на ноутбуке:  
  width = 380  
  height = 760  
  
Положение треугольника Game Starter:  
x + 172, y + 481, x + 209, y + 519  
  
Положение счёта:  
x + 269, y + 654, x + 304, y + 689

Двухзначный счет:  
- левая цифра: (6, 9, 16, 25)  
- правая цифра: (19, 9, 29, 25)  

Однозначный счет:
- цифра: (13, 9, 23, 25)  

Трехзначный счет:
- левая цифра: (4, 12, 12, 23)  
- средняя цифра: (14, 12, 22, 23)  
- правая цифра: (23, 12, 31, 23)  

Тарелки:  
- размер: 58x58  
- x центра первой тарелки: x + 18 + 29
- y центра второй тарелки: y + 229 + 29
- горизонтальная маржа: 0
- вертикальная маржа: 4


In [4]:
# Initialize parameters
parameters = {}

# Game Starter
parameters["Game_Starter"] = np.array(Image.open("Game_Starter.png"))

# Ethalon Single Digit
tmp = []

for f in glob.glob("Ethalon_Single\\S*.png"):
    img = io.imread(f)
    tmp.append(img)
    parameters["Ethalon_Single"] = np.array(tmp)
    
# Ethalon Large Digits
tmp = []

for f in glob.glob("Ethalon_Large\\L*.png"):
    img = io.imread(f)
    tmp.append(img)

parameters["Ethalon_Large_Left"] = np.array(tmp)

tmp = []
    
for f in glob.glob("Ethalon_Large\\R*.png"):
    img = io.imread(f)
    tmp.append(img)

parameters["Ethalon_Large_Right"] = np.array(tmp)

# Ethalon Small Digits
tmp = []

for f in glob.glob("Ethalon_Small\\L*.png"):
    img = io.imread(f)
    tmp.append(img)

parameters["Ethalon_Small_Left"] = np.array(tmp)

tmp = []

for f in glob.glob("Ethalon_Small\\M*.png"):
    img = io.imread(f)
    tmp.append(img)

parameters["Ethalon_Small_Middle"] = np.array(tmp)

tmp = []

for f in glob.glob("Ethalon_Small\\R*.png"):
    img = io.imread(f)
    tmp.append(img)

parameters["Ethalon_Small_Right"] = np.array(tmp)

# Load ethalon colors: RGB+Purple
with open('colors.pickle', 'rb') as f:
    colors = pickle.load(f)
    
parameters["Ethalon_Colors"] = colors

In [26]:
#
# Выравниваем приложение на экране
#
aeroflot_x, aeroflot_y, aeroflot_width, aeroflot_height = fix_window_coordinates_and_size("FRD L19", use_external_screen=True)

assert ((aeroflot_width == 380) and (aeroflot_height == 760)), print("Incorrect window size!")


In [5]:
# Тут будем хранить историю
#replay_memory = []
#with open('replay_memory_1D.pickle', 'rb') as f:
#   replay_memory = pickle.load(f)

# Число возможных ходов
number_of_moves = 71

In [6]:
# Загрузим ранее созданную модель
aero_cnn = load_model('AeroCNN_v4.h5')

In [7]:
# Количество предсказаний
total_num_of_predictions = 0
num_of_successful_predictions = 0

In [519]:
# Готовим модель
aero_cnn = Aero_CNN_v4(input_shape=(7, 6, 1), classes=number_of_moves)
aero_cnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [29]:
# Максимальный размер памяти
max_memory_size = 50000

# Вероятность выбора случайного хода
epsilon = 0.9

# Размер минибатча
minibatch = 32

# Пенальти за будущие ходы
gamma = 0.2

# Основной цикл, в котором играем 10 игр
for game in range(10):
    if (check_if_new_game(aeroflot_x, aeroflot_y, parameters)):
        # Press the Start New Game button
        press_start(aeroflot_x, aeroflot_y)
        win32api.Sleep(1000)
    else:
        print("Something is wrong. Exiting...")
        break

    # Делаем 50 ходов. Последний ход нужно будет обработать отдельно. С ним есть нюанс
    moves_left = 50
    game_score = 0
    reward = 0

    # Взбодрим генератор случайных чисел
    seed()

    # Делаем 50 ходов
    while (moves_left > 0):
        # Смотрим картинку перед ходом
        s_before = pixelization_v2(aeroflot_x, aeroflot_y, parameters)

        # Выбираем ход
        prob = uniform(0, 1)

        if (prob > epsilon):
            # Будем делать неслучайный ход
            if (total_num_of_predictions == 50):
                print(">>> Successful predictions ratio:", num_of_successful_predictions / total_num_of_predictions)
                total_num_of_predictions = 0
                num_of_successful_predictions = 0
            
            wise_move_debug_flag = 1
            predictions = aero_cnn.predict(np.expand_dims(s_before, axis=0))
            move = np.argmax(predictions) + 1
            wise_pred = predictions.max()
            total_num_of_predictions = total_num_of_predictions + 1
        else:
            # Случайный ход
            wise_move_debug_flag = 0
            move = randint(1, number_of_moves)

        # Делаем собственно ход
        make_move(aeroflot_x, aeroflot_y, number_of_moves, move)

        if (moves_left > 1):
            picture_1 = pixelization_v2(aeroflot_x, aeroflot_y, parameters)
            win32api.Sleep(1000)
            picture_2 = pixelization_v2(aeroflot_x, aeroflot_y, parameters)

            while (np.array_equal(picture_1, picture_2) == False):
                picture_1 = pixelization_v2(aeroflot_x, aeroflot_y, parameters)
                win32api.Sleep(500)
                picture_2 = pixelization_v2(aeroflot_x, aeroflot_y, parameters)

            # Определяем очки за ход - результат хода
            current_score = get_current_score(aeroflot_x, aeroflot_y, parameters)

            if (current_score != -1):
                if ((game_score == 0) and (current_score == 10)):
                        current_score = 0

                reward = current_score - game_score
                game_score = current_score

            # Смотрим картинку после хода
            s_after = picture_2

        else:
            # Это последний пятидесятый ход, его обрабатываем особенным образом
            # Тут проблема в том, что экран потухнет, и будет надпись об окончании игры
            # Нужно успеть вытащить счет и скрин с финальной разблюдовкой до этого момента
            #
            print("Handling last move...")
            last_move = []
            counter = 50

            picture_1 = pixelization_v2(aeroflot_x, aeroflot_y, parameters)
            current_score = get_current_score(aeroflot_x, aeroflot_y, parameters)

            while ((current_score != -1) and (counter > 0)):
                last_move.append((current_score, picture_1))

                picture_1 = pixelization_v2(aeroflot_x, aeroflot_y, parameters)
                current_score = get_current_score(aeroflot_x, aeroflot_y, parameters)

                counter = counter - 1

            # Берем финальный счет и финальную картинку
            if (counter < 50):
                current_score = last_move[-1][0]
                reward = current_score - game_score      
                s_after = last_move[-1][1]
            else:
                reward = 0

        # Если ход прошел без ошибок, то добавляем ход в память
        if (current_score != -1):
            if (len(replay_memory) == max_memory_size):
                replay_memory.pop(0)

            replay_memory.append((s_before, move, reward, s_after))     
            
            # DEBUG: отладочная информация по неслучайным ходам
            if (wise_move_debug_flag == 1):
                if (reward > 0):
                    num_of_successful_predictions = num_of_successful_predictions + 1
                print("Non-random move {}. Predicted {}, actual reward {}".format(move, round(wise_pred, 3), reward))
                
            # Конструируем минибатч
            if (len(replay_memory) > minibatch):
                if (reward == 0):
                    # Обязательно включаем ход с нулевым reward в обучение, чтобы избежать зацикливания предсказаний на нём
                    samples = sample(replay_memory[:-1], minibatch - 1)
                    samples.append(replay_memory[-1])
                else:
                    samples = sample(replay_memory, minibatch)
            else:
                samples = replay_memory
                
            #
            # Обучаем сеть
            #
            # Строим X_train
            X_train_now = ()
            X_train_future = ()

            for s in range(minibatch):
                X_train_now = X_train_now + (samples[s][0],)
                X_train_future = X_train_future + (samples[s][3],)

            X_train_now = np.stack(X_train_now, axis=0)
            X_train_future = np.stack(X_train_future, axis=0)

            # Строим Y_train. На выходе размерность [minibatch, number_of_moves]
            Y_train_now = aero_cnn.predict(X_train_now)
            Y_train_future = aero_cnn.predict(X_train_future)

            for s in range(minibatch):
                actn = samples[s][1]
                rwrd = samples[s][2]
                if (rwrd > 0):
                    Y_train_now[s, actn - 1] = rwrd + gamma * Y_train_future[s, :].max()
                else:
                    # Если ход нулевой, то предсказывать 0 сразу!
                    Y_train_now[s, actn - 1] = 0

            # Делаем градиентный спуск, подкручиваем веса сети
            aero_cnn.fit(X_train_now, Y_train_now, epochs=15, verbose=0)
        else:
            print("DEBUG: Skipping bad move...")





        epsilon = epsilon - 0.001
        moves_left = moves_left - 1

    print("Game {} over.".format(game))
    print("---")
    win32api.Sleep(5000)
    
    # Запускаем новую игру, т.е. нажимаем на кнопку "Еще раз"
    press_next_game(aeroflot_x, aeroflot_y)
    win32api.Sleep(2000)

We can start a new game
Non-random move 59. Predicted 8.38599967956543, actual reward 0
Non-random move 49. Predicted 11.286999702453613, actual reward 0
Non-random move 19. Predicted 9.340999603271484, actual reward 4
Non-random move 8. Predicted 9.081000328063965, actual reward 0
Non-random move 31. Predicted 11.041999816894531, actual reward 16
Non-random move 48. Predicted 8.015000343322754, actual reward 0
Non-random move 14. Predicted 7.913000106811523, actual reward 0
Non-random move 10. Predicted 6.385000228881836, actual reward 0
Non-random move 15. Predicted 8.597000122070312, actual reward 4
Handling last move...
Game 0 over.
---
We can start a new game
Non-random move 49. Predicted 7.71999979019165, actual reward 0
Non-random move 44. Predicted 6.965000152587891, actual reward 0
Non-random move 61. Predicted 8.045999526977539, actual reward 0
Non-random move 49. Predicted 7.690999984741211, actual reward 0
Non-random move 10. Predicted 7.377999782562256, actual reward 3
Han

Non-random move 9. Predicted 4.243000030517578, actual reward 0
Non-random move 54. Predicted 3.9049999713897705, actual reward 0
Non-random move 23. Predicted 5.203000068664551, actual reward 3
Non-random move 13. Predicted 8.46500015258789, actual reward 3
Non-random move 41. Predicted 10.781000137329102, actual reward 0
Non-random move 43. Predicted 8.692999839782715, actual reward 0
Non-random move 40. Predicted 7.164000034332275, actual reward 0
Non-random move 42. Predicted 6.958000183105469, actual reward 0
Handling last move...
Non-random move 54. Predicted 6.89900016784668, actual reward 0
Game 7 over.
---
We can start a new game
Non-random move 48. Predicted 9.904999732971191, actual reward 0
Non-random move 22. Predicted 9.569999694824219, actual reward 5
Non-random move 22. Predicted 9.916999816894531, actual reward 3
Non-random move 16. Predicted 10.630999565124512, actual reward 0
Non-random move 22. Predicted 8.531999588012695, actual reward 3
Non-random move 50. Predict

In [11]:
#
# Very Important!!!
#
aero_cnn.save("AeroCNN_v4.h5")

with open('replay_memory_1D.pickle', 'wb') as f:
    pickle.dump(replay_memory, f)

In [30]:
len(replay_memory)

5264

In [464]:
total_num_of_predictions

11

In [526]:
for layer in aero_cnn.layers:
    if (layer.name == "conv0"):
        lst = layer.get_weights()
        break


In [539]:
filters = lst[0]
filters[:, :, :, 11].reshape(3, 3)
#filters[:, :, :, 0].reshape(3, 3, 4)

array([[-0.01659712, -0.11228048, -0.01099141],
       [-0.2804085 ,  0.1258426 ,  0.257772  ],
       [ 0.1437774 ,  0.01205748, -0.09395663]], dtype=float32)

In [506]:
aero_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 7, 6, 1)           0         
_________________________________________________________________
zero_padding2d_19 (ZeroPaddi (None, 9, 8, 1)           0         
_________________________________________________________________
conv0 (Conv2D)               (None, 7, 6, 32)          320       
_________________________________________________________________
activation_33 (Activation)   (None, 7, 6, 32)          0         
_________________________________________________________________
conv1 (Conv2D)               (None, 5, 4, 64)          18496     
_________________________________________________________________
activation_34 (Activation)   (None, 5, 4, 64)          0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 1280)              0         
__________

In [27]:
%%time
get_current_score(aeroflot_x, aeroflot_y, parameters)

Wall time: 83 ms


35

In [358]:
results[1][1].shape

(7, 6, 4)

In [510]:
%%time
picture_1 = pixelization_v2(aeroflot_x, aeroflot_y, parameters)

Wall time: 76 ms


In [512]:
picture_1.shape

(7, 6)

In [548]:
picture_1[5, :]

array([[ 1.],
       [ 0.],
       [ 0.],
       [ 3.],
       [ 2.],
       [ 2.]])

In [406]:
X_train = results[1][1]
X_train = np.expand_dims(X_train, axis=0)
Y_train = np.zeros((1, 142))

for rr in range(50):
    jj = randint(0, 141)
    Y_train[0, jj] = randint(1, 9)

In [373]:
aero_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 7, 6, 4)           0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 11, 10, 4)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 7, 6, 8)           808       
_________________________________________________________________
activation_3 (Activation)    (None, 7, 6, 8)           0         
_________________________________________________________________
conv1 (Conv2D)               (None, 5, 4, 16)          1168      
_________________________________________________________________
activation_4 (Activation)    (None, 5, 4, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 320)               0         
__________

In [405]:
predictions

array([[  6.53922844,   0.        ,   0.        ,   7.80634499,
          7.30447292,   3.78879094,  11.14229012,  11.93689919,
          4.98499155,   5.99331045,   7.56705952,   4.97728157,
          7.87488127,   0.        ,  13.27569866,   6.92039585,
          0.        ,   4.96320534,   8.07557678,  11.78556919,
         13.18387508,   9.10807705,   5.56552172,   7.45903015,
         15.27207088,   6.32335901,   4.55224133,   5.45091486,
          9.23403835,   5.23410654,   5.97049475,   2.63473272,
          0.        ,   7.08221531,   3.95775652,   7.56428432,
          6.17845583,   3.91580534,   7.89203548,   3.77617788,
          0.        ,   5.40313768,   6.6367588 ,   7.64022684,
          9.9065752 ,   8.51534081,   8.67140293,   4.50644112,
          9.50450325,   0.        ,   8.16275883,  15.68370533,
          6.00693178,   9.38634014,  14.23967934,   7.04720116,
          5.5672493 ,   5.11613989,   7.88436174,  10.19739342,
          5.74646759,   2.77500272,   7.

In [110]:
for move, pred in enumerate(predictions.reshape(-1)):
    if (pred > 0):
        print(move + 1, pred)

In [132]:
process_move_71(38)

(1, 2, 'right')

In [471]:
tmp = pixelization_v2(aeroflot_x, aeroflot_y, parameters)

In [473]:
tmp.reshape(7, 6)

array([[ 0.25,  0.5 ,  0.75,  0.25,  1.  ,  0.25],
       [ 1.  ,  0.5 ,  1.  ,  0.5 ,  1.  ,  1.  ],
       [ 0.75,  1.  ,  0.5 ,  0.25,  0.25,  0.5 ],
       [ 1.  ,  0.25,  0.5 ,  1.  ,  0.25,  0.5 ],
       [ 0.25,  0.75,  0.25,  1.  ,  1.  ,  0.75],
       [ 1.  ,  0.5 ,  1.  ,  0.5 ,  0.5 ,  0.75],
       [ 1.  ,  0.75,  0.5 ,  0.5 ,  1.  ,  1.  ]])

In [489]:
cnt = len(replay_memory)

for item in range(1770, cnt):
    xxx = (replay_memory[item][0] + 1) / 4.0
    yyy = (replay_memory[item][3] + 1) / 4.0
    
    tmp.append( (xxx, replay_memory[item][1], replay_memory[item][2], yyy) )

In [492]:
tmp[-1][3].reshape(7, 6)

array([[ 1.  ,  0.75,  1.  ,  1.  ,  0.75,  1.  ],
       [ 0.5 ,  0.25,  0.75,  0.75,  0.5 ,  0.5 ],
       [ 0.5 ,  0.75,  0.75,  0.5 ,  1.  ,  0.75],
       [ 0.75,  0.5 ,  0.25,  0.75,  1.  ,  0.75],
       [ 0.75,  0.75,  0.25,  0.5 ,  0.75,  1.  ],
       [ 0.25,  0.5 ,  0.5 ,  0.75,  0.75,  1.  ],
       [ 0.75,  1.  ,  0.25,  0.5 ,  0.5 ,  0.25]])

In [515]:
# Готовим модель
tmp = Aero_CNN_v4(input_shape=(7, 6, 1), classes=number_of_moves)
tmp.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [516]:
tmp.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 7, 6, 1)           0         
_________________________________________________________________
zero_padding2d_22 (ZeroPaddi (None, 9, 8, 1)           0         
_________________________________________________________________
conv0 (Conv2D)               (None, 7, 6, 32)          320       
_________________________________________________________________
activation_40 (Activation)   (None, 7, 6, 32)          0         
_________________________________________________________________
conv1 (Conv2D)               (None, 5, 4, 64)          18496     
_________________________________________________________________
activation_41 (Activation)   (None, 5, 4, 64)          0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 1280)              0         
__________

In [10]:
%%time

seed()

# Пенальти за будущие ходы
gamma = 0.2

for i in range(20000):
    samples = sample(replay_memory, 32)
    
    # Строим X_train
    X_train_now = ()
    X_train_future = ()

    for s in range(32):
        X_train_now = X_train_now + (samples[s][0],)
        X_train_future = X_train_future + (samples[s][3],)

    X_train_now = np.stack(X_train_now, axis=0)
    X_train_future = np.stack(X_train_future, axis=0)

    # Строим Y_train. На выходе размерность [minibatch, number_of_moves]
    Y_train_now = aero_cnn.predict(X_train_now)
    Y_train_future = aero_cnn.predict(X_train_future)

    for s in range(32):
        actn = samples[s][1]
        rwrd = samples[s][2]
        if (rwrd > 0):
            Y_train_now[s, actn - 1] = rwrd + gamma * Y_train_future[s, :].max()
        else:
            # Если ход нулевой, то предсказывать 0 сразу!
            Y_train_now[s, actn - 1] = 0

    # Делаем градиентный спуск, подкручиваем веса сети
    aero_cnn.fit(X_train_now, Y_train_now, epochs=15, verbose=0)


Wall time: 57min 10s


In [560]:
get_current_score(aeroflot_x, aeroflot_y, parameters)

3325


-1

In [565]:
ret = check_if_new_game(aeroflot_x, aeroflot_y, parameters)

In [589]:
ret[7, 5:-5]

array([161, 161, 161, 161, 161, 161, 241, 254, 252, 249, 171, 161, 159,
       140, 140, 140, 140, 140, 140, 140, 140, 140, 140, 140, 140, 140, 140], dtype=uint8)

In [590]:
parameters["Game_Starter"][7, 5:-5]

array([161, 161, 161, 161, 162, 161, 175, 251, 252, 216, 163, 161, 161,
       145, 140, 140, 140, 140, 140, 140, 140, 140, 140, 140, 140, 140, 140], dtype=uint8)